----

# Store location analysis
#### Business Problem - An upcoming coffee supply chain company with direct access to coffee plantations wants to open its own stores in London. Before taking the first step to open its first store, the company would like to understand the market in general.

Approach - Foursquare API will be used to gather the location data and use it for analysis.
 - We will use the search query as 'Coffee' to procure the data for the current cafes in London
 - Additional search queries can be used to further study the reviews and ratings of the cafe to understand the general sentiment
 - Based on density of the cafes, a competitive landscape can be considered and leveraged to decide where to position the new cafe

In [169]:
#Importing necessary libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [170]:
# FourSquare API

CLIENT_ID = 'XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU'
CLIENT_SECRET = 'PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2' 
VERSION = '20190301'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU
CLIENT_SECRET:PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2


In [171]:
#Obtaining the required co-ordinates

address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5073219 -0.1276474


In [172]:
search_query = 'Coffee'
radius = 2000

In [173]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU&client_secret=PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2&ll=51.5073219,-0.1276474&v=20190301&query=Coffee&radius=2000&limit=30'

In [174]:
results = requests.get(url).json()

In [175]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4ac518edf964a520c1ac20e3,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,Monmouth Coffee Company,v-1585556317,NaN
1,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4b4b47f5f964a520609626e3,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,Costa Coffee,v-1585556317,NaN
2,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4cda944022bd721e533df047,15 Regent St,GB,London,United Kingdom,Jermyn St,462,"[15 Regent St (Jermyn St), London, Greater Lon...","[{'lat': 51.50879033505443, 'label': 'display'...",51.508790,-0.133887,NaN,SW1Y 4LR,Greater London,Costa Coffee,v-1585556317,NaN
3,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4b2630f7f964a5200a7924e3,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,Costa Coffee,v-1585556317,NaN
4,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,551a93a2498ed90a2cc7e544,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,Department of Coffee and Social Affairs,v-1585556317,NaN


In [176]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3
2,Costa Coffee,Coffee Shop,15 Regent St,GB,London,United Kingdom,Jermyn St,462,"[15 Regent St (Jermyn St), London, Greater Lon...","[{'lat': 51.50879033505443, 'label': 'display'...",51.508790,-0.133887,NaN,SW1Y 4LR,Greater London,4cda944022bd721e533df047
3,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3
4,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544


In [177]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3
2,Costa Coffee,Coffee Shop,15 Regent St,GB,London,United Kingdom,Jermyn St,462,"[15 Regent St (Jermyn St), London, Greater Lon...","[{'lat': 51.50879033505443, 'label': 'display'...",51.508790,-0.133887,NaN,SW1Y 4LR,Greater London,4cda944022bd721e533df047
3,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3
4,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544
5,Costa Coffee,Coffee Shop,17 Embankment Pl,GB,London,United Kingdom,NaN,336,"[17 Embankment Pl, London, Greater London, WC2...","[{'lat': 51.50730858076348, 'label': 'display'...",51.507309,-0.122793,Embankmen,WC2N 6NN,Greater London,4ae815e0f964a52014ae21e3
6,Ralph's Coffee & Bar,Café,173 Regent St,GB,London,United Kingdom,NaN,1009,"[173 Regent St, London, Greater London, W1B 4J...","[{'lat': 51.51219425144683, 'label': 'display'...",51.512194,-0.139929,NaN,W1B 4JQ,Greater London,5883913551d19e062a758a77
7,Coffee Geek and Friends,Coffee Shop,Unit 22 Cardinal Place,GB,London,United Kingdom,6 Cathedral Walk,1444,"[Unit 22 Cardinal Place (6 Cathedral Walk), Lo...","[{'lat': 51.497637766011685, 'label': 'display...",51.497638,-0.141523,NaN,SW1E 5JH,Greater London,5565c24d498e2885d1b9a54e
8,Crosstown Doughnuts & Coffee,Donut Shop,4 Broadwick St,GB,London,United Kingdom,NaN,889,"[4 Broadwick St, London, Greater London, W1F 8...","[{'lat': 51.514042566300574, 'label': 'display...",51.514043,-0.134604,"Soho, London, Greater London",W1F 8HJ,Greater London,553e368d498ecddb31302a1c
9,Costa Coffee,Coffee Shop,226 High Holborn,GB,London,United Kingdom,at Kingsway,1253,"[226 High Holborn (at Kingsway), London, Great...","[{'lat': 51.517563, 'label': 'display', 'lng':...",51.517563,-0.120122,NaN,WC1V 7DZ,Greater London,4ba0a226f964a520237537e3


In [178]:
# Obtaining the ratings for the dataframe

rating=[]

for venue in dataframe_filtered['id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    print(result)
    try:
        rating.append(result['response']['venue']['rating'])
    except:
        rating.append(0.0)

{'meta': {'code': 200, 'requestId': '5e81aa4add0f850028142b47'}, 'response': {'venue': {'listed': {'count': 3626, 'groups': [{'count': 3626, 'name': 'Lists from other people', 'items': [{'name': "London's best coffee shops", 'id': '4f3b8d2be4b04bfb5a0ec433', 'updatedAt': 1334755613, 'followers': {'count': 750}, 'logView': True, 'createdAt': 1329302827, 'listItems': {'count': 25, 'items': [{'id': 'v4ac518edf964a520c1ac20e3', 'createdAt': 1329490642}]}, 'type': 'others', 'user': {'type': 'chain', 'id': '10403171', 'firstName': 'Time Out London', 'photo': {'suffix': '/BX4LJ4AWOITAKFEP.png', 'prefix': 'https://fastly.4sqi.net/img/user/'}}, 'photo': {'visibility': 'public', 'id': '4ea2982b775b5c3e83346218', 'width': 612, 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/KJFT4G53CMFMBYMIEAF0BGH2VBGNSPLZGSUQ0B30R3WHR4AH.jpg', 'createdAt': 1319278635, 'user': {'id': '1657188', 'firstName': 'Paul in', 'photo': {'suffix': '/1657188-OEUO2M1UNJARL4W5.jpg', 'prefix': 'https://fastly.4sq

In [179]:
rating

[9.2,
 4.7,
 6.2,
 6.5,
 7.7,
 6.5,
 9.0,
 8.0,
 8.3,
 5.7,
 6.5,
 6.1,
 5.1,
 9.0,
 6.0,
 8.7,
 6.2,
 6.7,
 8.7,
 6.8,
 5.8,
 0.0,
 8.7,
 8.6,
 5.6,
 6.8,
 6.5,
 6.3,
 6.6,
 0.0]

In [180]:
# Adding the rating list procured to the dataframe

dataframe_filtered['rating']=rating
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,rating
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3,9.2
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3,4.7
2,Costa Coffee,Coffee Shop,15 Regent St,GB,London,United Kingdom,Jermyn St,462,"[15 Regent St (Jermyn St), London, Greater Lon...","[{'lat': 51.50879033505443, 'label': 'display'...",51.508790,-0.133887,NaN,SW1Y 4LR,Greater London,4cda944022bd721e533df047,6.2
3,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3,6.5
4,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544,7.7


---

## Exploratory Analysis

Visualizing the coffee shops in a 2 km radius

In [181]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)


# add the center as a red circle mark
folium.features.RegularPolygonMarker([latitude, longitude],
    radius=10,
    popup='Center',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add the selected coffee spots to the map as blue markers
for lat, lng, label1, label2 in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name, dataframe_filtered.rating):
    folium.features.RegularPolygonMarker(
        [lat, lng],
        radius=5,
        popup=label2,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### Findings:
- Towards north-east at Hatton Garden, there are 2 highest rated restaurants
- Oxford Circus has the maximum density of coffee shops giving customers plenty of options with some high rated
- There are 3 coffee shops at Kingsway near St.Giles that have below average ratings; the same goes for Picadilly circus


- A recommendation could be to open a new coffee shop around the vicinity of Picadilly circus or Kingsway which are closer to the central point and the comptetive landscape seems favorable

---

#### Exploring openly available data sources to help in our aim to triangulate a vicinity for opening a coffee shop

Population density across boroughs in London procured from https://data.london.gov.uk/dataset/land-area-and-population-density-ward-and-borough :

In [182]:
df = pd.read_csv(r'C:\Users\Ritwik\Desktop\housing-density-borough.csv')

df = df[df.Year==2020]

df.head()

,Code,Name,Year,Source,Population,Inland_Area _Hectares,Total_Area_Hectares,Population_per_hectare,Square_Kilometres,Population_per_square_kilometre
21,E09000001,City of London,2020,GLA Population Projections,8046,290.4,314.9,27.7,2.9,2770.7
73,E09000002,Barking and Dagenham,2020,GLA Population Projections,218367,3610.8,3779.9,60.5,36.1,6047.6
125,E09000003,Barnet,2020,GLA Population Projections,407148,8674.8,8674.8,46.9,86.7,4693.4
177,E09000004,Bexley,2020,GLA Population Projections,254970,6058.1,6428.6,42.1,60.6,4208.8
229,E09000005,Brent,2020,GLA Population Projections,343853,4323.3,4323.3,79.5,43.2,7953.6


In [183]:
# Creating a general London map

london_map = folium.Map(location=[latitude, longitude], zoom_start=12)

london_map

Searching for the London GeoJSON file - https://joshuaboyd1.carto.com/tables/london_boroughs_proper/public

In [184]:
# Import geoJSON file for London
# To create a Choropleth map, we need a GeoJSON file that defines the areas/boundaries of the location we are interested in.
london_geo = r'C:\Users\Ritwik\Desktop\london_boroughs_proper.geojson'

In [185]:
# generate choropleth map using the population in df

london_map.choropleth(
    geo_data=london_geo,
    data=df,
    columns=['Name', 'Population_per_square_kilometre'],
    key_on='feature.properties.name', #check the json file to get the districts tag
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population density'
)

# display map
london_map

C:\Users\Ritwik\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Findings:
-The stores selected from out initial search query lie in the second most densely populated region in London helping vouch our selection of areas (Kingsway, Picadilly) for opening a coffee shop
-Our interest of Borough is Camden

---

This part of the analysis assumes that the first store has been launched in the selected Borough:

As a second stage in expansion we would want to find out demographics similar to Camden borough, we will try to cluster the Boroughs based on certain demographics

London Borough data can be found in : https://data.london.gov.uk/dataset/london-borough-profiles

## K-Means Clustering

In [148]:
import random 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

In [150]:
dfdem = pd.read_csv(r'C:\Users\Ritwik\Desktop\london-borough-profiles.csv')

dfdem.head()

,Area name,GLA Population Estimate 2017,"Proportion of population of working-age, 2015","Proportion of population aged 65 and over, 2015",Employment rate (%) (2015),Number of jobs by workplace (2014),"Jobs Density, 2015","Number of active businesses, 2015",Two-year business survival rates (started in 2013)
0,City of London,8800,73.1,15.5,64.6,500400,84.3,26130,64.3
1,Barking and Dagenham,209000,63.1,9.7,65.8,58900,0.5,6560,73.0
2,Barnet,389600,64.9,14.0,68.5,167300,0.7,26190,73.8
3,Bexley,244300,62.9,16.6,75.1,80700,0.6,9075,73.5
4,Brent,332100,67.8,11.3,69.5,133600,0.6,15745,74.4


In [151]:
#Normalizing values over the standard deviation

from sklearn.preprocessing import StandardScaler
X = dfdem.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

C:\Users\Ritwik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Ritwik\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ -3.48822671e+00,   1.25789205e+00,   1.21589846e+00,
         -1.95312289e+00,   2.43846370e+00,   5.64991070e+00,
          1.11751732e+00,  -2.79183716e+00],
       [ -7.91290656e-01,  -1.33813282e+00,  -6.80784223e-01,
         -1.66431159e+00,  -8.26873648e-01,  -2.05601443e-01,
         -1.13092140e+00,  -2.26920525e-01],
       [  1.64160970e+00,  -8.70848344e-01,   7.25377076e-01,
         -1.01448616e+00,  -2.51464258e-02,  -1.91626473e-01,
          1.12441084e+00,   8.93387892e-03],
       [ -3.15756976e-01,  -1.39005332e+00,   1.57561414e+00,
          5.73975994e-01,  -6.65640682e-01,  -1.98613958e-01,
         -8.41967725e-01,  -7.95115224e-02],
       [  8.67015179e-01,  -1.18001131e-01,  -1.57561414e-01,
         -7.73810075e-01,  -2.74391881e-01,  -1.98613958e-01,
         -7.56373098e-02,   1.85824682e-01],
       [  8.10436101e-01,  -1.46793407e+00,   1.86992697e+00,
          6.22111211e-01,  -3.17288725e-01,  -1.98613958e-01,
         -8.13819156e-02,   1.4

In [165]:
clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[3 0 2 0 2 2 4 2 2 2 0 2 0 0 0 0 4 2 4 0 0 2 2 0 2 2 0 4 0 4 0 2 1]


In [166]:
dfdem['Cluster'] = labels

dfdem

,Area name,GLA Population Estimate 2017,"Proportion of population of working-age, 2015","Proportion of population aged 65 and over, 2015",Employment rate (%) (2015),Number of jobs by workplace (2014),"Jobs Density, 2015","Number of active businesses, 2015",Two-year business survival rates (started in 2013),Cluster
0,City of London,8800,73.1,15.5,64.6,500400,84.3,26130,64.3,3
1,Barking and Dagenham,209000,63.1,9.7,65.8,58900,0.5,6560,73.0,0
2,Barnet,389600,64.9,14.0,68.5,167300,0.7,26190,73.8,2
3,Bexley,244300,62.9,16.6,75.1,80700,0.6,9075,73.5,0
4,Brent,332100,67.8,11.3,69.5,133600,0.6,15745,74.4,2
5,Bromley,327900,62.6,17.5,75.3,127800,0.6,15695,78.6,2
6,Camden,242500,71.0,11.7,69.2,377400,2.2,31385,73.6,4
7,Croydon,386500,64.9,13.0,75.4,141600,0.6,15540,75.3,2
8,Ealing,351600,66.8,11.8,72.7,160500,0.7,18700,75.8,2
9,Enfield,333000,64.4,12.8,73.0,128800,0.6,13925,74.2,2


In [168]:
dfexp = dfdem[dfdem.Cluster == 4]
dfexp

,Area name,GLA Population Estimate 2017,"Proportion of population of working-age, 2015","Proportion of population aged 65 and over, 2015",Employment rate (%) (2015),Number of jobs by workplace (2014),"Jobs Density, 2015","Number of active businesses, 2015",Two-year business survival rates (started in 2013),Cluster
6,Camden,242500,71.0,11.7,69.2,377400,2.2,31385,73.6,4
16,Hillingdon,301000,65.6,13.1,73.2,220900,1.0,13505,75.0,4
18,Islington,231200,75.3,8.8,72.6,231500,1.5,22110,72.5,4
27,Southwark,314300,73.5,7.9,74.2,285900,1.3,17120,73.4,4
29,Tower Hamlets,304000,73.9,6.0,70.4,280100,1.4,18390,69.7,4


## Discussion

1. Out initial launch is based on the population density and competitive landscape, this is assuming real-estate prices are not under stringent consideration. Higher visibility is of utmost importance.
2. The next step of clustering is illustrative and depends on the initial success of the first coffee shop launch. The clusters can be used to either replicate the success of the first store or explore and test new demographics

## Conclusion

 1. This analysis is purely based on location and dempgraphics and can be seen as a part of a larger analytical scope to figure out the best way to launch a coffee chain, it goes without saying that it is iterative
 2. Based on the above analysis, it is recommended to release a coffee shop in Camden Borough at Kingsway road - it has the most promising competitie landscape and promises the highest visibility

---